In [1]:
%matplotlib inline
import nivapy3 as nivapy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona

plt.style.use('ggplot')

# Checking "!000 Lakes" sampling locations are within lakes

NVE has previusoly supplied an ESRI File Geodatabase of Norwegian lakes.

In [2]:
# Get layers in GDB
gdb_path = r'../../NVEData.gdb'
layer_list = fiona.listlayers(gdb_path)
print(layer_list)

['Innsjo']


In [3]:
# Read lakes dataset
gdf = gpd.read_file(gdb_path, layer='Innsjo')
gdf.head()

,objektType,vatnLnr,navn,hoyde_moh,areal_km2,magasinNr,magasinKategori,magasinFormal_liste,vassdragsNr,elvenavnHierarki,...,arealNorge_km2,dybdekart,nedborfeltAreal_km2,dataUttaksdato,eksportType,SHAPE_Length,SHAPE_Area,UTM_Z33_E,UTM_Z33_N,geometry
0,Innsjø,165333,None,-8888,0.006160,NaN,NaN,None,025.H,KVINA,...,0.006160,None,0.010000,2017-09-27T00:00:00,NVEs nedlastningsløsning,394.219939,6159.97130,49828.230106,6.560515e+06,"(POLYGON ((49808.83000000007 6560467.3902, 498..."
1,Innsjø,56888,Juovsajavrit,438,0.217176,NaN,NaN,None,234.GBJ,IESJÅKKA/KARASJÅKKA/TANA,...,0.217176,None,5.104375,2017-09-27T00:00:00,NVEs nedlastningsløsning,4450.517347,217176.01480,864997.937388,7.764739e+06,"(POLYGON ((864712.04 7764341.240200002, 864706..."
2,Innsjø,151207,None,-8888,0.009965,NaN,NaN,None,109.5G,LITLEDALSELVA,...,0.009965,None,0.110000,2017-09-27T00:00:00,NVEs nedlastningsløsning,549.726466,9965.47710,169206.496645,6.939095e+06,(POLYGON ((169246.5300000003 6939027.540199999...
3,Innsjø,192275,None,-8888,0.010995,NaN,NaN,None,019.CF,GJØV/ARENDALSVASSDRAGET,...,0.010995,None,0.060000,2017-09-27T00:00:00,NVEs nedlastningsløsning,422.604515,10994.76345,98523.076095,6.574547e+06,(POLYGON ((98582.62999999989 6574576.120200001...
4,Innsjø,97273,None,-8888,0.008618,NaN,NaN,None,196.A4B,TAKELVA/MÅLSELVVASSDRAGET,...,0.008618,None,0.640000,2017-09-27T00:00:00,NVEs nedlastningsløsning,369.532207,8618.14750,657612.040353,7.673926e+06,(POLYGON ((657658.1200000001 7673975.150199998...


And here are the sampling locations from the 1995 survey. **Note:** There are three lakes in this dataset with missing co-ordinates in RESA.

In [5]:
# Read sampling pts
stn_path = r'../../1000_lakes_1995.xlsx'
stn_df = pd.read_excel(stn_path, sheet_name='DATA')
stn_df.head()

,Station ID,Station Code,Station name,Latitude,Longitude,Altitude
0,115,831-501,Brårvatn,59.294921,7.727118,902.0
1,116,1640-603,Tufsingen,62.612294,11.876558,781.0
2,125,2030-607,St.Valvatnet,69.694975,30.656211,162.0
3,135,1228-501,Steinavatn,59.859877,6.578783,1050.0
4,166,1018-4,Kleivsetvannet,58.119559,7.664899,93.0


In [12]:
# Link pts to polys
df = nivapy.spatial.identify_point_in_polygon(stn_df,
                                              gdf,
                                              pt_col='Station ID',
                                              poly_col='vatnLnr',
                                              lat_col='Latitude',
                                              lon_col='Longitude')

# Join NVE name
df = pd.merge(df, gdf[['vatnLnr', 'navn']],
              how='left', on='vatnLnr')
df.head()

,Station ID,Station Code,Station name,Latitude,Longitude,Altitude,vatnLnr,navn
0,115,831-501,Brårvatn,59.294921,7.727118,902.0,NaN,NaN
1,116,1640-603,Tufsingen,62.612294,11.876558,781.0,NaN,NaN
2,125,2030-607,St.Valvatnet,69.694975,30.656211,162.0,2474.0,Vađajávri
3,135,1228-501,Steinavatn,59.859877,6.578783,1050.0,NaN,NaN
4,166,1018-4,Kleivsetvannet,58.119559,7.664899,93.0,11592.0,Kleivsetvannet


In [13]:
# Which locations are not in a lake?
not_in_lake = df[pd.isna(df['vatnLnr'])]
print(len(not_in_lake), 'station locations are not within a lake.')
not_in_lake.head()

67 station locations are not within a lake.


,Station ID,Station Code,Station name,Latitude,Longitude,Altitude,vatnLnr,navn
0,115,831-501,Brårvatn,59.294921,7.727118,902.0,NaN,NaN
1,116,1640-603,Tufsingen,62.612294,11.876558,781.0,NaN,NaN
3,135,1228-501,Steinavatn,59.859877,6.578783,1050.0,NaN,NaN
25,26,604-3-3,Hengsvatnet,59.656902,9.443625,452.0,NaN,NaN
35,39,938-3-4,Storolavsvatnet,58.859362,7.262579,848.0,NaN,NaN
